In [4]:
#pip install folium
#!pip install geopandas

In [ ]:
import geopandas as gpd
from shapely.geometry import MultiLineString

# Cargar los archivos GeoJSON para Carvallo
#geojson_files = [
#    "./Carvallo/Zona_solapamiento_1_2.geojson",
#    "./Carvallo/Zona_solapamiento_3_4.geojson",
#    "./Carvallo/Zona_solapamiento_5_6.geojson",
#    "./Carvallo/Zona_solapamiento_7_8.geojson",
#    "./Carvallo/Zona_solapamiento_9_10.geojson"
#]

# Cargar los archivos GeoJSON para Ruben Dario
#geojson_files = [
#    "./Ruben_Dario/Zona_solapamiento_1_2.geojson",
#    "./Ruben_Dario/Zona_solapamiento_3_4.geojson",
#    "./Ruben_Dario/Zona_solapamiento_5_6.geojson",
#    "./Ruben_Dario/Zona_solapamiento_7_8.geojson",
#    "./Ruben_Dario/Zona_solapamiento_9_10.geojson"
#]

# Cargar los archivos GeoJSON para San Mateo
geojson_files = [
    "./San_Mateo/Zona_solapamiento_1_2.geojson",
    "./San_Mateo/Zona_solapamiento_3_4.geojson",
    "./San_Mateo/Zona_solapamiento_5_6.geojson",
    "./San_Mateo/Zona_solapamiento_7_8.geojson",
    "./San_Mateo/Zona_solapamiento_9_10.geojson"
]

# Cargar los archivos finales generados para cada zona

# geojson_files = [
#     "./Carvallo/Zonas_solapadas_final.geojson",
#     "./Ruben_Dario/Zonas_solapadas_final.geojson",
#     "./San_Mateo/Zona_solapamiento_final.geojson"

# ]



# Leer los archivos GeoJSON
gdfs = [gpd.read_file(f) for f in geojson_files]

# Asegurarse de que todos los GeoDataFrames tengan el mismo CRS
for gdf in gdfs:
    if gdf.crs is None:
        gdf.set_crs('EPSG:4326', allow_override=True, inplace=True)
    else:
        gdf.to_crs('EPSG:4326', inplace=True)

# Realizar la intersección entre los GeoDataFrames utilizando Shapely
from shapely.ops import unary_union

# Inicializamos la intersección con el primer GeoDataFrame
intersection_result = gdfs[0].geometry.unary_union

for gdf in gdfs[1:]:
    # Usamos unary_union para simplificar la geometría antes de la intersección
    intersection_result = intersection_result.intersection(unary_union(gdf.geometry))

# Verificar si hay algún solapamiento y exportar el resultado
if intersection_result.is_empty:
    print("No se encontraron solapamientos.")
else:
    # Convertir el resultado a un GeoDataFrame
    result_gdf = gpd.GeoDataFrame(geometry=[intersection_result], crs=gdfs[0].crs)
    result_gdf.to_file("zonas_solapadas_final.geojson", driver="GeoJSON")
    print("Solapamientos exportados a zonas_solapadas_final.geojson")





Solapamientos exportados a zonas_solapadas_final.geojson


C:\Users\jose.cordova\AppData\Local\Temp\ipykernel_29776\2088519434.py:56: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersection_result = gdfs[0].geometry.unary_union


In [ ]:
## Calcular el solapmiento final de los capas, dado los resultados preliminares generados

In [9]:
import geopandas as gpd
from shapely.geometry import MultiLineString, LineString
from shapely.ops import unary_union

# Archivos GeoJSON de entrada
geojson_files = [
    "./Carvallo/Zonas_solapadas_final.geojson",
    "./Ruben_Dario/Zonas_solapadas_final.geojson",
    "./San_Mateo/Zona_solapamiento_final.geojson"
]

# Leer y procesar los archivos GeoJSON
gdfs = []
for f in geojson_files:
    try:
        gdf = gpd.read_file(f)
        if gdf.crs is None:
            gdf.set_crs('EPSG:4326', allow_override=True, inplace=True)
        else:
            gdf.to_crs('EPSG:4326', inplace=True)

        # Filtrar solo líneas (evita problemas con geometrías inesperadas)
        gdf = gdf[gdf.geometry.type.isin(["LineString", "MultiLineString"])]
        
        if not gdf.empty:
            gdfs.append(gdf)
            print(f"📌 Cargado: {f} con {len(gdf)} líneas")
        else:
            print(f"⚠️ Advertencia: {f} no contiene líneas válidas.")

    except Exception as e:
        print(f"❌ Error al leer {f}: {e}")

# Verificar si hay capas cargadas
if len(gdfs) < 2:
    print("❌ Se necesitan al menos dos capas para calcular solapamientos.")
    exit()

# 🔍 **Calcular la intersección**
print("🔍 Calculando intersección...")

# Unir todas las geometrías en una sola
merged_lines = unary_union([gdf.geometry.unary_union for gdf in gdfs])

# Calcular intersección entre todas las capas
intersection_result = merged_lines
for gdf in gdfs:
    intersection_result = intersection_result.intersection(gdf.geometry.unary_union)

# Verificar si el resultado es válido
if intersection_result.is_empty:
    print("❌ No se encontraron solapamientos.")
    exit()

# Convertir a MultiLineString si es necesario
if intersection_result.geom_type == "LineString":
    intersection_result = MultiLineString([intersection_result])
elif intersection_result.geom_type != "MultiLineString":
    print(f"⚠️ Advertencia: Tipo de geometría inesperado ({intersection_result.geom_type}), puede que QGIS no lo muestre correctamente.")

# Crear un GeoDataFrame con atributos
result_gdf = gpd.GeoDataFrame({
    "ID": range(1, len(intersection_result.geoms) + 1),
    "Longitud_m": [line.length for line in intersection_result.geoms],
    "geometry": list(intersection_result.geoms)
}, crs="EPSG:4326")

# Exportar a GeoJSON
output_file = "zonas_solapadas_final.geojson"
if not result_gdf.empty and not result_gdf.geometry.is_empty.all():
    result_gdf.to_file(output_file, driver="GeoJSON")
    print(f"✅ Exportado correctamente a {output_file}")
else:
    print("❌ Error: No hay geometrías válidas para exportar.")


📌 Cargado: ./Carvallo/Zonas_solapadas_final.geojson con 1 líneas
📌 Cargado: ./Ruben_Dario/Zonas_solapadas_final.geojson con 1 líneas
📌 Cargado: ./San_Mateo/Zona_solapamiento_final.geojson con 2 líneas
🔍 Calculando intersección...
⚠️ Advertencia: Tipo de geometría inesperado (GeometryCollection), puede que QGIS no lo muestre correctamente.
✅ Exportado correctamente a zonas_solapadas_final.geojson


C:\Users\jose.cordova\AppData\Local\Temp\ipykernel_29776\1734878527.py:43: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  merged_lines = unary_union([gdf.geometry.unary_union for gdf in gdfs])
C:\Users\jose.cordova\AppData\Local\Temp\ipykernel_29776\1734878527.py:48: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersection_result = intersection_result.intersection(gdf.geometry.unary_union)
